In [84]:
import json
import pandas as pd
import copy
import numpy as np


In [157]:
with open('predict_properties/test_predictions.json', 'r') as res_file:
        results_dict = json.load(res_file)

with open('data/normalisation_stats.json', 'r') as norm_file:
        norm_dict = json.load(norm_file)
print(norm_dict)
base_values = pd.read_csv('data/test_values.csv')

{'ET30': {'mean': 41.78074074074074, 'std': 7.192808351716445}, 'alpha': {'mean': 0.24287037037037038, 'std': 0.4369842507179892}, 'beta': {'mean': 0.43583333333333335, 'std': 0.29472662765133556}, 'pi_star': {'mean': 0.5897058823529412, 'std': 0.2639473990589902}, 'SA': {'mean': 0.1259194630872483, 'std': 0.21698369962375624}, 'SB': {'mean': 0.41759060402684567, 'std': 0.30223225840275103}, 'SP': {'mean': 0.7266174496644294, 'std': 0.08636132434197896}, 'SdP': {'mean': 0.5100536912751678, 'std': 0.3397320642449683}, 'N_mol_cm3': {'mean': 0.009754802259887006, 'std': 0.004920819417443173}, 'n': {'mean': 1.4358654929577466, 'std': 0.06159673594301312}, 'fn': {'mean': 0.2591721408450704, 'std': 0.037422473958837625}, 'delta': {'mean': 20.727083333333333, 'std': 4.862952950986547}}


In [90]:
def avg_predictions(dictionary):
    predictions_averaged = {}

    for smiles, properties in dictionary.items():
        predictions_averaged[smiles] = {}

        for prop, indices in properties.items():
            total_sum = 0
            total_count = 0
            
            # Calculate the sum and count of all predictions
            for index, prediction_list in indices.items():
                total_sum += sum(prediction_list)
                total_count += len(prediction_list)
            
            # Calculate the average, handling the case where total_count is zero to prevent division by zero
            if total_count > 0:
                average_prediction = total_sum / total_count
            else:
                average_prediction = 0 # or None, depending on desired behavior
            
            predictions_averaged[smiles][prop] = average_prediction
    return predictions_averaged
    
predictions_averaged = avg_predictions(results_dict)

In [ ]:
true_values_dict = {}

# Use df.itertuples() for a memory-efficient way to iterate over DataFrame rows.
# 'index=False' prevents the row index from being included in the tuple.
for row in base_values.itertuples(index=False):
    # The first element of the tuple is the SMILES string
    smiles = row[1]
    
    # Initialize a new dictionary for this SMILES if it doesn't exist
    if smiles not in true_values_dict:
        true_values_dict[smiles] = {}
        
    # Iterate through the rest of the columns to get the properties and values
    # We slice the row tuple from the second element (index 1) onwards.
    # We also get the corresponding column names from df.columns, excluding 'SMILES'.
    for prop_name, value in zip(base_values.columns[1:], row[1:]):
        # Store the value in the nested dictionary
        true_values_dict[smiles][prop_name] = value


In [89]:
def unnormalize_dict(data_dict, stats):
    """
    Unnormalizes the values in a nested dictionary using z-score statistics.

    Args:
        data_dict (dict): The nested dictionary with normalized values.
                          Format: {smiles: {property: value}}
        stats (dict): The dictionary containing mean and std for each property.
                      Format: {property: {'mean': value, 'std': value}}

    Returns:
        dict: A new dictionary with unnormalized values.
    """
    unnormalized_data = copy.deepcopy(data_dict)
    
    # Iterate through each smiles string in the dictionary
    for smiles, properties in unnormalized_data.items():
        # Iterate through each property and its normalized value
        for prop, value in properties.items():
            try:
                mean = stats[prop]['mean']
                std = stats[prop]['std']
                
                # Apply the reverse z-score formula: x = (z * std) + mean
                unnormalized_value = (value * std) + mean
                
                # Update the value in the new dictionary
                unnormalized_data[smiles][prop] = unnormalized_value
            except KeyError:
                print(f"Warning: Statistics not found for property '{prop}'. Skipping unnormalization for this property.")
                # If stats are not found, we keep the original value
                unnormalized_data[smiles][prop] = value
            
    return unnormalized_data

In [91]:
prediction_avg_unnorm = unnormalize_dict(predictions_averaged, norm_dict)
true_values_dict_unorm = unnormalize_dict(true_values_dict, norm_dict)
print(true_values_dict_unorm.keys())

dict_keys(['CCCCC', 'C1CCOC1', 'CC(=O)c1ccccc1', 'CCOC=O', 'CCCCO'])


In [165]:
import pandas as pd

def mse_dataframe_with_avgs(pred_dict, true_dict):
    # Compute MSE table
    data = {}
    for smiles, pred_props in pred_dict.items():
        data[smiles] = {}
        for prop, pred_value in pred_props.items():
            true_value = true_dict[smiles][prop]
            mse = (pred_value - true_value) ** 2
            data[smiles][prop] = mse
    df = pd.DataFrame.from_dict(data, orient='index')
    
    # Add per-row mean (average MSE per solvent)
    df["Average_per_solvent"] = df.mean(axis=1)
    
    # Add per-column mean (average MSE per property)
    avg_row = df.mean(axis=0)
    
    # The intersection of averages = overall average MSE
    avg_row["Average_per_solvent"] = avg_row.mean()
    
    # Append the averages row
    df.loc["Average_per_property"] = avg_row
    
    return df

# Example usage:
df_mse = mse_dataframe_with_avgs(prediction_avg_unnorm, true_values_dict_unorm)
#print(df_mse)
decoder_MSE = df_mse.iloc[5].copy()
print(decoder_MSE)

delta                  0.744454
SdP                    0.015733
SB                     0.001985
SA                     0.000095
fn                     0.000070
n                      0.000268
SP                     0.001164
N_mol_cm3              0.000001
ET30                   1.245248
beta                   0.008403
pi_star                0.017029
alpha                  0.000978
Average_per_solvent    0.169619
Name: Average_per_property, dtype: float64


In [167]:
with open('predict_properties/template_preds.json', 'r') as template_file:
        temp_dict = json.load(template_file)
temp_avg = avg_predictions(temp_dict)

temp_unnorm = unnormalize_dict(temp_avg, norm_dict)
df_mse_temp = mse_dataframe_with_avgs(temp_unnorm, true_values_dict_unorm)
print(df_mse_temp)

                         delta       SdP        SB        SA            fn  \
C1CCOC1               0.177700  0.056302  0.000602  0.000235  1.564750e-04   
CCCCO                 0.331666  0.016388  0.003761  0.000034  2.495663e-07   
CCCCC                 1.189663  0.005479  0.002056  0.000014  1.892873e-04   
CCOC=O                2.121827  0.001238  0.003511  0.000086  7.772237e-07   
CC(=O)c1ccccc1        0.035833  0.000376  0.000010  0.000085  1.573190e-05   
Average_per_property  0.771338  0.015957  0.001988  0.000091  7.250421e-05   

                                 n        SP     N_mol_cm3      ET30  \
C1CCOC1               6.605138e-04  0.000232  5.851446e-09  0.003651   
CCCCO                 8.447491e-08  0.000322  1.037829e-06  0.272274   
CCCCC                 6.461181e-04  0.005230  3.219535e-07  1.575431   
CCOC=O                4.884155e-05  0.000029  2.501931e-06  0.029536   
CC(=O)c1ccccc1        1.740383e-05  0.000072  3.034920e-06  4.232485   
Average_per_property 

In [194]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

def compare_predictions_by_n(dict_1, dict_2, ground_truth=None,
                             label1='Dataset 1', label2='Dataset 2', output_dir='plots'):
    """
    Create plots comparing predictions for each property from two different datasets,
    with optional ground truth values plotted as horizontal lines.

    Args:
        json_file_path1 (str): Path to the first JSON file containing the data
        json_file_path2 (str): Path to the second JSON file containing the data
        ground_truth (dict): Dictionary mapping property names to their ground truth values
        label1 (str): Label for the first dataset (default: 'Dataset 1')
        label2 (str): Label for the second dataset (default: 'Dataset 2')
        output_dir (str): Directory to save the plots (default: 'plots')
    """
    # Load data from both JSON files
    data1 = dict_1
    data2 = dict_2
    plt.rcParams['font.size'] = 18

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get all property names from both datasets
    all_properties = set(data1.keys()) | set(data2.keys())

    # Process each property
    for property_name in all_properties:
        plt.figure(figsize=(12, 8))

        # Initialize variables to track x-axis range
        all_n_values = []

        # Process first dataset if property exists
        if property_name in data1:
            property_data1 = data1[property_name]

            # Prepare data for plotting
            n_values1 = []
            means1 = []
            std_devs1 = []
            mins1 = []
            maxs1 = []

            # Sort n values numerically
            sorted_ns1 = sorted(map(int, property_data1.keys()))

            for n in sorted_ns1:
                n_str = str(n)
                values = property_data1[n_str]
                n_values1.append(n)
                means1.append(np.mean(values))
                std_devs1.append(np.std(values))
                mins1.append(min(values))
                maxs1.append(max(values))

            all_n_values.extend(n_values1)

            # Plot first dataset
            plt.errorbar(n_values1, means1, yerr=std_devs1, fmt='-o',
                         capsize=5, capthick=5, label=f'{label1} Mean ± Std Dev',
                         color="#dacb00", alpha=0.8)

            plt.fill_between(n_values1, mins1, maxs1, alpha=0.50,
                             label=f'{label1} Min/Max Range', color='#dacb00')

        # Process second dataset if property exists
        if property_name in data2:
            property_data2 = data2[property_name]

            # Prepare data for plotting
            n_values2 = []
            means2 = []
            std_devs2 = []
            mins2 = []
            maxs2 = []

            # Sort n values numerically
            sorted_ns2 = sorted(map(int, property_data2.keys()))

            for n in sorted_ns2:
                n_str = str(n)
                values = property_data2[n_str]
                n_values2.append(n)
                means2.append(np.mean(values))
                std_devs2.append(np.std(values))
                mins2.append(min(values))
                maxs2.append(max(values))

            all_n_values.extend(n_values2)

            # Plot second dataset
            plt.errorbar(n_values2, means2, yerr=std_devs2, fmt='-s',
                         capsize=5, capthick=5, label=f'{label2} Mean ± Std Dev',
                         color="#010055", alpha=0.8)

            plt.fill_between(n_values2, mins2, maxs2, alpha=0.50,
                             label=f'{label2} Min/Max Range', color='#010055')

        # Plot ground truth if provided
        if ground_truth and property_name in ground_truth:
            # Get the range of x values to draw the line across the entire plot
            x_range = sorted(set(all_n_values))
            if x_range:
                x_min, x_max = min(x_range), max(x_range)
                # Add some padding to the line
                x_padding =  0.5 #(x_max - x_min) * 0.05 if x_max > x_min else
                x_line = [x_min - x_padding, x_max + x_padding]
                y_line = [ground_truth[property_name], ground_truth[property_name]]

                plt.plot(x_line, y_line, '--', color="#005ac0", linewidth=5,
                         label=f'Ground Truth ({ground_truth[property_name]:.3f})', alpha=0.8)

        # Handle case where property only exists in one dataset
        if property_name not in data1:
            print(f"Warning: Property '{property_name}' not found in {dict_1}")
        elif property_name not in data2:
            print(f"Warning: Property '{property_name}' not found in {dict_2}")

        # Handle case where ground truth is provided but property not found
        if ground_truth and property_name not in ground_truth:
            print(f"Warning: Ground truth value for '{property_name}' not provided")

        # Customize plot
        title = f'Prediction Comparison for {property_name} by Position n'
        if ground_truth and property_name in ground_truth:
            title += f' (Ground Truth: {ground_truth[property_name]:.3f})'
        plt.title(title)
        plt.xlabel('Prediction Position (n)')
        plt.ylabel('Prediction Value')

        # Set x-ticks to show all n values from both datasets
        unique_n_values = sorted(set(all_n_values))
        if unique_n_values:
            plt.xticks(unique_n_values)
        plt.grid(True, linestyle='--', alpha=0.6)

        # Place the legend below the plot
        plt.legend(bbox_to_anchor=(0.5, -0.25), loc='upper center', ncol=2)
        
        plt.tight_layout()

        # Save plot
        plot_filename = os.path.join(output_dir, f'{property_name}_comparison.png')
        plt.savefig(plot_filename, bbox_inches='tight', dpi=300)
        plt.close()

        print(f'Saved comparison plot for {property_name} to {plot_filename}')

In [195]:
import copy

def unnormalize_dict_with_lists(data_dict, stats):
    """
    Unnormalizes the values in a nested dictionary using z-score statistics.
    Handles single values and lists of values.

    Args:
        data_dict (dict): The nested dictionary with normalized values.
                          Format: {smiles: {property: value or [values]}}
        stats (dict): The dictionary containing mean and std for each property.
                      Format: {property: {'mean': value, 'std': value}}

    Returns:
        dict: A new dictionary with unnormalized values.
    """
    unnormalized_data = copy.deepcopy(data_dict)
    
    for smiles, properties in unnormalized_data.items():
        for prop, indicies in properties.items():
            for index, values in indicies.items():
                try:
                    mean = stats[prop]['mean']
                    std = stats[prop]['std']
                    # Check if the value is a list and iterate if so
                    if isinstance(values, list):
                        unnormalized_data[smiles][prop][index] = [
                            (item * std) + mean for item in values
                        ]
                    
                    else:
                        # Apply the reverse z-score formula for a single value
                        unnormalized_value = (values * std) + mean
                        unnormalized_data[smiles][prop][index] = unnormalized_value

                except KeyError:
                    print(f"Warning: Statistics not found for property '{prop}'. Skipping.")
                    # The original value (single or list) is kept
                    unnormalized_data[smiles][prop] = value

    return unnormalized_data

In [196]:
temp_unorm = unnormalize_dict_with_lists(temp_dict, norm_dict)
results_unnorm = unnormalize_dict_with_lists(results_dict, norm_dict) 

compare_predictions_by_n(temp_unorm['CC(=O)c1ccccc1'],results_unnorm['CC(=O)c1ccccc1'],

                      ground_truth=true_values_dict_unorm['CC(=O)c1ccccc1'], label1='template predictions', label2='scratch predictions')

Saved comparison plot for fn to plots\fn_comparison.png
Saved comparison plot for alpha to plots\alpha_comparison.png
Saved comparison plot for delta to plots\delta_comparison.png
Saved comparison plot for SP to plots\SP_comparison.png
Saved comparison plot for SB to plots\SB_comparison.png
Saved comparison plot for n to plots\n_comparison.png
Saved comparison plot for N_mol_cm3 to plots\N_mol_cm3_comparison.png
Saved comparison plot for SA to plots\SA_comparison.png
Saved comparison plot for ET30 to plots\ET30_comparison.png
Saved comparison plot for pi_star to plots\pi_star_comparison.png
Saved comparison plot for beta to plots\beta_comparison.png
Saved comparison plot for SdP to plots\SdP_comparison.png


In [158]:
gp_preds = pd.read_csv('predict_properties\predictions_gp_mean.csv')
rf_preds = pd.read_csv('predict_properties\predictions_rf_mean.csv')
print(gp_preds.columns)
print(gp_preds['Canonical Solvent SMILES'])
def fix_columns(df):
    col_list = df.columns.to_list()
    new_colums = []
    for i in col_list:
        new_colums.append(i.split(" ")[-1])
    new_df = pd.DataFrame()
    new_df[new_colums] = df[col_list]
    return new_df

new_gp = fix_columns(gp_preds)
new_rf = fix_columns(rf_preds)


print(new_gp.columns)

Index(['Predicted ET30', 'Predicted alpha', 'Predicted beta',
       'Predicted pi_star', 'Predicted SA', 'Predicted SB', 'Predicted SP',
       'Predicted SdP', 'Predicted N_mol_cm3', 'Predicted n', 'Predicted fn',
       'Predicted delta', 'Canonical Solvent SMILES'],
      dtype='object')
0             CCCCC
1           C1CCOC1
2    CC(=O)c1ccccc1
3            CCOC=O
4             CCCCO
Name: Canonical Solvent SMILES, dtype: object
Index(['ET30', 'alpha', 'beta', 'pi_star', 'SA', 'SB', 'SP', 'SdP',
       'N_mol_cm3', 'n', 'fn', 'delta', 'SMILES'],
      dtype='object')


In [162]:
def unnormalize_data(df, normalization_dict):
    """
    Unnormalizes the data in a pandas DataFrame using a dictionary of mean and std values.

    Args:
        df (pd.DataFrame): The DataFrame with normalized values.
        normalization_dict (dict): A dictionary where keys are column names (properties)
                                   and values are dictionaries containing 'mean' and 'std' keys.
                                   Example: {'property_a': {'mean': 10, 'std': 2}}

    Returns:
        pd.DataFrame: A new DataFrame with the unnormalized values.
    """
    # Create a copy of the DataFrame to avoid modifying the original one
    unnormalized_df = df.copy()

    # Iterate through each column in the DataFrame
    for column in unnormalized_df.columns:
        # Check if the column exists in the normalization dictionary
        if column in normalization_dict:
            # Retrieve the mean and standard deviation for the current column
            mean_val = normalization_dict[column]['mean']
            std_val = normalization_dict[column]['std']
            
            # Unnormalize the column using the formula: (normalized_value * std) + mean
            unnormalized_df[column] = (unnormalized_df[column] * std_val) + mean_val
        else:
            # Optionally print a warning for columns not found in the dictionary
            print(f"Warning: Column '{column}' not found in the normalization dictionary. Skipping unnormalization for this column.")
    
    return unnormalized_df

unnorm_new_gp = unnormalize_data(new_gp, norm_dict)
unnorm_new_rf = unnormalize_data(new_rf, norm_dict)
unnorm_base_values = unnormalize_data(base_values, norm_dict )

print(norm_dict['delta'].items())
print(new_gp['delta'])
print(unnorm_new_gp['delta'])
print(base_values['delta'])
print(unnorm_base_values['delta'])

dict_items([('mean', 20.727083333333333), ('std', 4.862952950986547)])
0   -1.024847
1    0.024913
2   -0.254717
3    0.011764
4    0.431490
Name: delta, dtype: float64
0    15.743301
1    20.848235
2    19.488406
3    20.784292
4    22.825397
Name: delta, dtype: float64
0   -1.280515
1   -0.355151
2    0.014994
3   -0.334588
4    0.529085
Name: delta, dtype: float64
0    14.5
1    19.0
2    20.8
3    19.1
4    23.3
Name: delta, dtype: float64


In [166]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# Debug function to examine data before MSE calculation
def debug_data_before_mse(df_prediction, df_true, columns_to_compare, name=""):
    print(f"\n=== DEBUG: {name} ===")
    print(f"Prediction df shape: {df_prediction.shape}")
    print(f"True df shape: {df_true.shape}")
    
    # Merge to see what we're actually comparing
    merged_df = pd.merge(df_prediction, df_true, on='SMILES', how='inner', suffixes=('_pred', '_true'))
    print(f"Merged df shape: {merged_df.shape}")
    
    for col in ['ET30', 'delta']:  # Focus on problematic columns
        if col in columns_to_compare:
            pred_col = f"{col}_pred"
            true_col = f"{col}_true"
            
            if pred_col in merged_df.columns and true_col in merged_df.columns:
                pred_vals = merged_df[pred_col].dropna()
                true_vals = merged_df[true_col].dropna()
                
                print(f"\n{col}:")
                print(f"  Predicted - mean: {pred_vals.mean():.6f}, std: {pred_vals.std():.6f}, range: [{pred_vals.min():.6f}, {pred_vals.max():.6f}]")
                print(f"  True      - mean: {true_vals.mean():.6f}, std: {true_vals.std():.6f}, range: [{true_vals.min():.6f}, {true_vals.max():.6f}]")
                
                # Show first few values
                common_idx = pred_vals.index.intersection(true_vals.index)[:3]
                if len(common_idx) > 0:
                    print(f"  First 3 comparisons:")
                    for idx in common_idx:
                        print(f"    Pred: {pred_vals.loc[idx]:.6f}, True: {true_vals.loc[idx]:.6f}, Diff²: {(pred_vals.loc[idx] - true_vals.loc[idx])**2:.6f}")

def calculate_mse_by_smiles_debug(df_prediction, df_true, columns_to_compare, debug_name=""):
    """
    Enhanced MSE calculation with debugging information
    """
    # Debug before processing
    debug_data_before_mse(df_prediction, df_true, columns_to_compare, debug_name)
    
    # Create copies to avoid modifying original DataFrames
    df_pred_copy = df_prediction.copy()
    df_true_copy = df_true.copy()
    
    # Merge the two DataFrames on the 'SMILES' column
    merged_df = pd.merge(
        df_pred_copy,
        df_true_copy,
        on='SMILES',
        how='inner',
        suffixes=('_pred', '_true')
    )

    if merged_df.empty:
        print("Warning: No matching 'SMILES' strings found in the two DataFrames.")
        return None

    mse_results = {}
    
    for column_name in columns_to_compare:
        prediction_column = f"{column_name}_pred"
        true_column = f"{column_name}_true"

        if prediction_column not in merged_df.columns or true_column not in merged_df.columns:
            print(f"Error: The specified column '{column_name}' was not found in both DataFrames. Skipping.")
            continue

        # Extract values and handle NaN
        y_pred = merged_df[prediction_column].dropna()
        y_true = merged_df[true_column].dropna()
        
        # Ensure both series have the same index
        common_index = y_pred.index.intersection(y_true.index)
        if len(common_index) == 0:
            print(f"Warning: No valid data points for column '{column_name}'.")
            continue
            
        y_pred = y_pred.loc[common_index]
        y_true = y_true.loc[common_index]

        # Calculate MSE
        mse = mean_squared_error(y_true, y_pred)
        mse_results[column_name] = mse
        
        # Extra debug for problematic columns
        if column_name in ['ET30', 'delta'] and mse > 1000:
            print(f"\n*** HIGH MSE WARNING for {column_name}: {mse:.2f} ***")
            print(f"Number of data points: {len(y_pred)}")
            print(f"Max absolute difference: {np.abs(y_pred - y_true).max():.6f}")

    if mse_results:
        mse_df = pd.DataFrame.from_dict(mse_results, orient='index', columns=['MSE'])
        mse_df.index.name = 'Column'
        return mse_df
    else:
        print("Warning: No MSE results calculated.")
        return None

# Your existing code with debug versions
compare_cols = ['ET30', 'alpha', 'beta', 'pi_star', 'SA', 'SB', 'SP', 'SdP', 'N_mol_cm3', 'n', 'fn', 'delta']

print("=== CHECKING DATA SHAPES AND BASIC INFO ===")
print(f"base_values shape: {base_values.shape}")
print(f"unnorm_base_values shape: {unnorm_base_values.shape}")

# Check if unnorm_base_values has the expected ranges
print(f"\nunnorm_base_values ET30 range: [{unnorm_base_values['ET30'].min():.2f}, {unnorm_base_values['ET30'].max():.2f}]")
print(f"unnorm_base_values delta range: [{unnorm_base_values['delta'].min():.2f}, {unnorm_base_values['delta'].max():.2f}]")

gp_mses = calculate_mse_by_smiles_debug(unnorm_new_gp, unnorm_base_values, compare_cols, "GP")
print("\nGP MSEs:")
print(gp_mses)

rf_mses = calculate_mse_by_smiles_debug(unnorm_new_rf, unnorm_base_values, compare_cols, "RF")
print("\nRF MSEs:")
print(rf_mses)

# Fix the df_avg creation
df_avg = pd.DataFrame()

# Use unnorm_base_values length instead of base_values
num_rows = len(unnorm_base_values)  # This was likely the issue!

for column in norm_dict.keys():
    if column in compare_cols:
        df_avg[column] = [norm_dict[column]['mean']] * num_rows

print(f"\nCorrected df_avg shape: {df_avg.shape}")

# Use unnorm_base_values SMILES
df_avg['SMILES'] = unnorm_base_values["SMILES"].values

avg_mses = calculate_mse_by_smiles_debug(df_avg, unnorm_base_values, compare_cols, "Average")
print('\nAverage MSEs:')
print(avg_mses)

# Additional diagnostic: Check if normalization dict makes sense
print(f"\n=== NORMALIZATION DICT CHECK ===")
for col in ['ET30', 'delta']:
    if col in norm_dict:
        print(f"{col} - mean: {norm_dict[col]['mean']:.6f}")
        # Compare with actual data range
        actual_mean = unnorm_base_values[col].mean()
        print(f"{col} - actual data mean: {actual_mean:.6f}")
        print(f"{col} - difference: {abs(norm_dict[col]['mean'] - actual_mean):.6f}")

print('DECODER MSE', decoder_MSE)

=== CHECKING DATA SHAPES AND BASIC INFO ===
base_values shape: (5, 14)
unnorm_base_values shape: (5, 14)

unnorm_base_values ET30 range: [31.00, 49.70]
unnorm_base_values delta range: [14.50, 23.30]

=== DEBUG: GP ===
Prediction df shape: (5, 13)
True df shape: (5, 14)
Merged df shape: (5, 26)

ET30:
  Predicted - mean: 40.190584, std: 5.812396, range: [32.551032, 48.736266]
  True      - mean: 39.920000, std: 6.763653, range: [31.000000, 49.700000]
  First 3 comparisons:
    Pred: 32.551032, True: 31.000000, Diff²: 2.405701
    Pred: 38.531386, True: 37.400000, Diff²: 1.280033
    Pred: 41.162043, True: 40.600000, Diff²: 0.315893

delta:
  Predicted - mean: 19.937926, std: 2.630562, range: [15.743301, 22.825397]
  True      - mean: 19.340000, std: 3.217608, range: [14.500000, 23.300000]
  First 3 comparisons:
    Pred: 15.743301, True: 14.500000, Diff²: 1.545798
    Pred: 20.848235, True: 19.000000, Diff²: 3.415974
    Pred: 19.488406, True: 20.800000, Diff²: 1.720278

GP MSEs:
      